In [1]:
%%time
!pip install -q -U langchain pypdf faiss-gpu
!pip install -q -U InstructorEmbedding sentence_transformers
!pip install huggingface_hub -q
!pip install gradio -q
!pip install watermark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.9/277.9 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%load_ext watermark

In [3]:
%watermark

Last updated: 2023-12-19T11:36:53.136377+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.58+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [5]:
import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

# loaders
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

# prompts
from langchain import PromptTemplate, LLMChain

# vector stores
from langchain.vectorstores import FAISS

# models
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

#retrievers
from langchain.chains import RetrievalQA

import torch


In [6]:
%watermark --iversions

torch    : 2.1.0+cu121
langchain: 0.0.351




**CFG**
class that enables easy and organized experimentation



In [32]:
class CFG:
  #LLMS
  model_name = 'mistralai/Mistral-7B-Instruct-v0.1'
  temperature = 0.5
  top_p = 0.95
  repetition_penalty = 1.15
  do_sample  = True
  max_new_tokens = 1024
  num_return_sequence =1

  # splitting
  split_chunk_size = 800
  split_overlap = 0

  # embeddings
  embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'

  #similar passages
  k = 5
  #paths
  PDFs_path = '/content/pdfs'
  Embeddings_path = '/content/faiss_index_hp'
  Persist_directory = ''


In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohammedelqajjam","key":"3ff7aee4f9db10c3e3206b2ab5e6ca74"}'}

In [14]:
# import kaggle
# from kaggle_secrets import UserSecretClient
# user_secrets = UserSecretClient()
huggingfacehub_api_token = "xxxxxxxxxxxxxxxxxx"

In [15]:
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id = CFG.model_name,
    model_kwargs={
      "max_new_tokens": CFG.max_new_tokens,
      "temperature": CFG.temperature,
      "top_p": CFG.top_p,
      "repetition_penalty": CFG.repetition_penalty,
      "do_sample": CFG.do_sample,
      "num_return_sequence": CFG.num_return_sequence
    },
    huggingfacehub_api_token = huggingfacehub_api_token
)

In [16]:
print(llm("""<s>[INST] Give me 5 examples of cool potions and explain what they do. [/INST]""", raw_response=True).strip())

Sure! Here are five examples of cool potions and their effects:

1. **Love Potion:** This potion is said to make the drinker fall deeply in love with the first person they see after consuming it. It's often portrayed as a romantic item, but its effectiveness varies greatly depending on the story or movie.
2. **Invisibility Potion:** As the name suggests, this potion makes the drinker invisible. It allows them to go unnoticed by others and can be useful for various purposes such as espionage or sneaking around. However, invisibility often has its drawbacks; for example, the person becomes intangible and can walk through walls or objects.
3. **Super Strength Potion:** This potion grants the drinker superhuman strength, allowing them to lift heavy objects and fight off enemies with ease. It's a popular choice among heroes and villains alike.
4. **Potion of Poison:** This potion is used to kill or harm others. When ingested or applied topically, it causes severe illness or death. It's impo

In [21]:
%%time
loader =DirectoryLoader(
    CFG.PDFs_path,
    glob = "./*.pdf",
    loader_cls=PyPDFLoader,
    show_progress=True,
    use_multithreading=True

)
documents = loader.load()


100%|██████████| 7/7 [01:25<00:00, 12.22s/it]

CPU times: user 1min 24s, sys: 1.11 s, total: 1min 25s
Wall time: 1min 25s


In [22]:
print(f'we have{len(documents)} pages in total')

we have4114 pages in total


In [24]:
documents[8].page_content

"8Ron\nP.S. Percy's Head Boy. He got the letter last week.Harry glanced back at the photograph. Percy, who was in his seventh and\nfinal year at Hogwarts, was looking particularly smug. He had pinned hisHead Boy badge to the fez perched jauntily on top of his neat hair, hishorn-rimmed glasses flashing in the Egyptian sun.\nHarry now turned to his present and unwrapped it. Inside was what looked\nlike a miniature glass spinning top. There was another note from Ronbeneath it.\nHarry -- this is a Pocket Sneakoscope. If there's someone untrustworthy\naround, it's supposed to light up and spin. Bill says it's rubbish soldfor wizard tourists and isn't reliable, because it kept lighting up atdinner last night. But he didn't realize Fred and George had put beetlesin his soup.\nBye --RonHarry put the Pocket Sneakoscope on his bedside table, where it stood\nquite still, balanced on its point, reflecting the luminous hands of hisclock. He looked at it happily for a few seconds, then picked up the

In [26]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = CFG.split_chunk_size,
    chunk_overlap = CFG.split_overlap
)
texts = text_splitter.split_documents(documents)
len(texts)

10519

In [31]:
%%time
#download embeddings model
embeddings = HuggingFaceInstructEmbeddings(
    model_name = CFG.embeddings_model_repo,
    model_kwargs = {"device": "cuda"}
)

#create embeddings and DB
vectordb = FAISS.from_documents(
    documents = texts,
    embedding = embeddings
)
#persist vector database
vectordb.save_local("faiss_index_hp")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512
CPU times: user 30.9 s, sys: 970 ms, total: 31.8 s
Wall time: 31.3 s


In [33]:
%%time
# load vector DB embeddings
vectordb = FAISS.load_local(
    CFG.Embeddings_path,
    embeddings
)


CPU times: user 49.2 ms, sys: 15 ms, total: 64.2 ms
Wall time: 63.4 ms


In [35]:
#test if vector Db was loaded correctly
vectordb.similarity_search('lord voldemort')

[Document(page_content='created the mask of ‘Lord Voldemort’ behind which he \nhas been hidden for so long.', metadata={'source': '/content/pdfs/Harry Potter - Book 6 - The Half-Blood Prince.pdf', 'page': 309}),
 Document(page_content='to save him from Lord Voldemort’s wrath.”\n682', metadata={'source': '/content/pdfs/Harry Potter - Book 7 - The Deathly Hallows.pdf', 'page': 689}),
 Document(page_content='And he saw himself, possessor of the Hallows, facing Voldemort,\n429', metadata={'source': '/content/pdfs/Harry Potter - Book 7 - The Deathly Hallows.pdf', 'page': 436}),
 Document(page_content='him -- he tried to take on Voldemort himself, to give my mum time to runfor it...."', metadata={'source': '/content/pdfs/Harry Potter - Book 3 - The Prisoner of Azkaban.pdf', 'page': 199})]

In [36]:
#we get the context part by embedding retrieval
prompt_template = """<s>[INST] You are given the context after <<CONTEXT>> and a question after <<QUESTION>>.

Answer the question by only using the information in context. Only base your answer on the information in the context. Even if you know something more,
keep silent about it. It is important that you only tell what can be infered from the context alone.

<<QUESTION>>{question}\n<<CONTEXT>>{context} [/INST]"""

PROMPT = PromptTemplate(
    template = prompt_template,
    input_variables = ["question", "context"]
)

In [37]:
llm_chain = LLMChain(prompt= PROMPT, llm=llm)

In [38]:
answer =llm_chain.run(question= "GIve me 5 moments where harry was almost dead.", context="")

print(answer.strip())

Based on the provided context, there were no instances of Harry being almost dead mentioned or implied.


**Retriever chain**

In [41]:
retriever = vectordb.as_retriever(search_kwargs =
                                  {"k":CFG.k, "search_type":"similarity"})
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = retriever,
    chain_type_kwargs = {"prompt":PROMPT},
    return_source_documents = True,
    verbose = False
)

In [43]:
#testing MMR search
question = "whats the name of harry's best friends?"
vectordb.max_marginal_relevance_search(question, k=CFG.k)

[Document(page_content='this morning. Yours sincerely, Harry Potter.  \nEven inside his head th e words sounded stupid. \nAnd so he tried to imagine his ot her best friend, Ron Weasley’s,', metadata={'source': '/content/pdfs/Harry Potter - Book 4 - The Goblet of Fire.pdf', 'page': 36}),
 Document(page_content='“And then I called out ‘Who’s there?’ ” \n“You couldn’t have found out who it was without \nasking?” Harry asked her, slightly frustrated.', metadata={'source': '/content/pdfs/Harry Potter - Book 6 - The Half-Blood Prince.pdf', 'page': 607}),
 Document(page_content='that Harry’s name got into the Goblet of Fire? Voldemort was', metadata={'source': '/content/pdfs/Harry Potter - Book 4 - The Goblet of Fire.pdf', 'page': 298}),
 Document(page_content='When he reached the living room he found all three Dursleys. They\nwere dressed for traveling: Uncle Vernon in a fawn zip-up jacket,\nAunt Petunia in a neat salmon-colored coat, and Dudly, Harry’s\nlarge, blond, muscular cousin, in his

In [44]:
#testing similarity search
question = "Which are hagrid's favourite animals?"
vectordb.similarity_search(question, k=CFG.k)

[Document(page_content="Harry could sort of see what Hagrid meant. Once you got over the first\nshock of seeing something that was, half horse, half bird, you startedto appreciate the hippogriffs' gleaming coats, changing smoothly fromfeather to hair, each of them a different color: stormy gray, bronze,", metadata={'source': '/content/pdfs/Harry Potter - Book 3 - The Prisoner of Azkaban.pdf', 'page': 91}),
 Document(page_content='would warn Hagrid myself, but I am  banished — it would be unwise \nfor me to go too near the forest now — Hagrid has troubles enough, \nwithout a centaurs’ battle.” \n“But — what’s Hagrid attempting to do?” said Harry nervously. \nFirenze looked at Harry impassively. \n“Hagrid has recently rendered me a great service,” said Firenze,', metadata={'source': '/content/pdfs/Harry Potter - Book 5 - The Order of the Phoenix.pdf', 'page': 619}),
 Document(page_content='CHAPTER  THIRTEEN \n\x91 198 \x91 nothing better than a pet drag on, as Harry, Ron, and Hermione \n

In [45]:
#Post-process outputs
def wrap_text_preserve_newlines(text, width=700):
  lines = text.split('\n')

  wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
  wrapped_text = '\n'.join(wrapped_lines)
  return wrapped_text

def process_llm_response(llm_response):
  ans = wrap_text_preserve_newlines(llm_response['result'])

  sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4] + ' - page: ' + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )

  ans = ans + '\n\nSources: \n' + sources_used
  return ans

In [46]:
def llm_ans(query):
  start = time.time()
  llm_response = qa_chain(query)
  ans = process_llm_response(llm_response)
  end = time.time()

  time_elapsed = int(round(end - start, 0))
  time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
  return ans.strip()+ time_elapsed_str

In [49]:
query = "how did Tom became lord voldemort?"
print(llm_ans(query))

From the provided context, it seems that Tom Riddle was a brilliant student at Hogwarts, who disappeared after leaving the school. He then traveled far and wide and sank so deeply into the dark arts that when he resurfaced, he was barely recognizable as Lord Voldemort. The context does not provide any direct information on how Tom Riddle became Lord Voldemort, but rather implies that his transformation into Lord Voldemort was due to his involvement in the dark arts and his descent into evil.

Sources: 
Harry Potter - Book 2 - The Chamber of Secrets - page: 365 
Harry Potter - Book 6 - The Half-Blood Prince - page: 309 
Harry Potter - Book 7 - The Deathly Hallows - page: 436 
Harry Potter - Book 4 - The Goblet of Fire - page: 298 
Harry Potter - Book 7 - The Deathly Hallows - page: 438

Time elapsed: 2 s
